In [8]:
import os
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from transformers import pipeline

In [9]:
model_path = "./models"  
pipe = pipeline("image-classification", model=model_path)

In [10]:
print(pipe)

In [11]:
def detect_deepfake_from_image(image_array, score_threshold=0.5):
    temp = "temp_frame.jpg"
    cv2.imwrite(temp, image_array)

    results = pipe(temp)
    os.remove(temp) 
    max_score = 0
    label = "Unknown"

    for result in results:
        if result['score'] > max_score:
            max_score = result['score']
            label = result['label']

    if label == 'Fake' and max_score > score_threshold:
        return "Fake", max_score
    elif label == 'Real' and max_score > score_threshold:
        return "Real", max_score
    else:
        return "Unknown", max_score

In [12]:
def on_upload_change(change):
    if change['new']: 
        for file_info in change['new']:
            content = file_info['content']
            image_array = cv2.imdecode(np.frombuffer(content, np.uint8), cv2.IMREAD_COLOR)

            classification_result, confidence_score = detect_deepfake_from_image(image_array)

            print(f"Classification Result: {classification_result}")
            print(f"Confidence Score: {confidence_score:.2f}")

In [13]:
upload_button = widgets.FileUpload(
    accept='.jpg,.jpeg,.png,.mp4', 
    multiple=False  
)

In [17]:
upload_button.observe(on_upload_change, names='value')
display(upload_button)

FileUpload(value=({'name': 'fake img.png', 'type': 'image/png', 'size': 452955, 'content': <memory at 0x000002…

Classification Result: Real
Confidence Score: 1.00
Classification Result: Fake
Confidence Score: 0.96
